# Exemple de réseaux récurent prédisant du texte
## Utilisation de l'architecture GRU

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Quelques phrases simples
sentences = [
    "la goutte d'eau qui fait déborder le vase", 
    "Il n'y a pas de fumée sans feu", 
    "Il faut battre le fer tant qu'il est chaud", 
    "Il ne faut pas mettre tous ses oeufs dans le même panier", 
    "Il faut tourner sept fois sa langue dans sa bouche avant de parler", 
    "L'habit ne fait pas le moine", 
    "Il ne faut pas réveiller le chat qui dort", 
    "Il faut se méfier de l'eau qui dort", 
    "C'est l'hôpital qui se moque de la charité", 
    "Qui vole un oeuf vole un boeuf", 
    "Chercher midi à quatorze heures", 
    "Avoir un poil dans la main", 
    "Être dans de beaux draps", 
    "Avoir la tête dans les nuages", 
    "Mettre les pieds dans le plat"]

In [3]:
# analyse du texte
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print("nb de mots différents rencontrés :", total_words)

nb de mots différents rencontrés : 72


In [4]:
type(tokenizer.word_index)
#from dict to list 
liste = list(tokenizer.word_index.keys())
print("voici les premiers mots trouvés : ")
for i in range(10): print(f"({i+1}:'{liste[i]}')", end= ", ")
print()

voici les premiers mots trouvés : 
(1:'le'), (2:'il'), (3:'dans'), (4:'qui'), (5:'de'), (6:'faut'), (7:'la'), (8:'pas'), (9:'ne'), (10:'un'), 


In [5]:
# transformation des textes en vecteurs 
input_sequences = []
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
# calibrage des vecteurs pour qu'ils aient tous la même longueur
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [7]:
print("la phrase '", sentences[0], "' est traduite en plusieurs vecteurs :")
split = sentences[0].split()
for i in range(6):
    print(input_sequences[i], end=" -> '")
    for j in range(i+2):
        print(split[j], end=" ")
    print("'")

la phrase ' la goutte d'eau qui fait déborder le vase ' est traduite en plusieurs vecteurs :
[ 0  0  0  0  0  0  0  0  0  0  0  7 19] -> 'la goutte '
[ 0  0  0  0  0  0  0  0  0  0  7 19 20] -> 'la goutte d'eau '
[ 0  0  0  0  0  0  0  0  0  7 19 20  4] -> 'la goutte d'eau qui '
[ 0  0  0  0  0  0  0  0  7 19 20  4 11] -> 'la goutte d'eau qui fait '
[ 0  0  0  0  0  0  0  7 19 20  4 11 21] -> 'la goutte d'eau qui fait déborder '
[ 0  0  0  0  0  0  7 19 20  4 11 21  1] -> 'la goutte d'eau qui fait déborder le '


In [8]:
# creer les x (premieres valeurs de chaque vecteur)
X = input_sequences[:, :-1]
# creer les y (derniere valeur de chaque vecteur)
y = input_sequences[:, -1]

In [9]:
# chaque mot de sortie est représenté par un vecteur de 0, avec 1 correspondant à l'indice du mot
#donc le vecteur est aussi grand que le nb de mots trouvés
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [10]:
#creation du model
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
model.add(GRU(120, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

c:\Program Files\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [11]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:

# entrainer le modele
print("patienter 30s pendant l'entrainement...")
model.fit(X, y, epochs=300, verbose=0)

patienter 30s pendant l'entrainement...


In [13]:
# fonction pour prédire le mot suivant
def predict_next_word(start_text, next_words=1):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([start_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                start_text += " " + word
                break
    
    return start_text

In [17]:
# Tester le modèle avec quelques textes
start_texts = [
    "il faut tourner sept fois sa  ",
    "la goutte d'eau qui fait  ",
    "Qui vole un oeuf   ",
]

for text in start_texts:
    print(f"first part : {text}")
    nb = 5 if text.count(" ")<5 else 3
    print(f"prédictions de {nb} mots.")
    print(f"Prediction: {predict_next_word(text, nb)}")
    print("-" * 50)

first part : il faut tourner sept fois sa  
prédictions de 3 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: il faut tourner sept fois sa   langue dans sa
--------------------------------------------------
first part : la goutte d'eau qui fait  
prédictions de 3 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: la goutte d'eau qui fait   déborder le vase
--------------------------------------------------
first part : Qui vole un oeuf   
prédictions de 3 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: Qui vole un oeuf    vole un boeuf
--------------------------------------------------


In [16]:
# Tester le modèle avec quelques textes
start_texts = [
    "le boeuf ne fait pas   ",
    "il faut battre le moine   ",
]

for text in start_texts:
    print(f"first part : {text}")
    nb = 5 #if text.count(" ")<5 else 3
    print(f"prédictions de {nb} mots.")
    print(f"Prediction: {predict_next_word(text, nb)}")
    print("-" * 50)

first part : le boeuf ne fait pas   
prédictions de 5 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction: le boeuf ne fait pas    le moine tant qu'il est
--------------------------------------------------
first part : il faut battre le moine   
prédictions de 5 mots.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: il faut battre le moine    tant qu'il est chaud chaud
--------------------------------------------------
